In [1]:
# Natural Language Processing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Importing the dataset
df = pd.read_csv('sentiment.tsv', delimiter = '\t', header=None,names = ["sentiment", "text"])
df.head()

,sentiment,text
0,neg,"@jamielewislewis i cant believe it, it really ..."
1,pos,having a vodka tonic and looking forward to go...
2,pos,@ddlovatofans1neg1 Could you follow me please....
3,pos,@jordanknight for once.................. PLEAS...
4,neg,Had a dream about a walk in fast food resturau...


In [2]:
# This Python file uses the following encoding: utf-8
import re

# Hashtags
hash_regex = re.compile(r"#(\w+)")
def hash_repl(match):
    return '__HASH_'+match.group(1).upper()    

# Handels
hndl_regex = re.compile(r"@(\w+)")
def hndl_repl(match):
    return '__HNDL'#_'+match.group(1).upper()    

# URLs
url_regex = re.compile(r"(http|https|ftp)://[a-zA-Z0-9\./]+")

# Spliting by word boundaries
word_bound_regex = re.compile(r"\W+")

# Repeating words like hurrrryyyyyy
rpt_regex = re.compile(r"(.)\1{1,}", re.IGNORECASE);
def rpt_repl(match):
    return match.group(1)+match.group(1)

# Emoticons
emoticons = \
    [('__EMOT_SMILEY',	[':-)', ':)', '(:', '(-:', ] )	,\
        ('__EMOT_LAUGH',		[':-D', ':D', 'X-D', 'XD', 'xD', ] )	,\
        ('__EMOT_LOVE',		['<3', ':\*', ] )	,\
        ('__EMOT_WINK',		[';-)', ';)', ';-D', ';D', '(;', '(-;', ] )	,\
        ('__EMOT_FROWN',		[':-(', ':(', '(:', '(-:', ] )	,\
        ('__EMOT_CRY',		[':,(', ':\'(', ':"(', ':(('] )	,\
    ]

#For emoticon regexes
def escape_paren(arr):
    return [text.replace(')', '[)}\]]').replace('(', '[({\[]') for text in arr]
def regex_union(arr):
    return '(' + '|'.join( arr ) + ')'
emoticons_regex = [ (repl, re.compile(regex_union(escape_paren(regx))) ) \
                    for (repl, regx) in emoticons ]

def processAll(text):

    text = re.sub( hash_regex, hash_repl, text )
    text = re.sub( hndl_regex, hndl_repl, text )
    text = re.sub( url_regex, ' __URL ', text )

    for (repl, regx) in emoticons_regex :
        text = re.sub(regx, ' '+repl+' ', text)

    text = text.replace('\'','')
    text = re.sub( word_bound_regex ,' ', text )
    text = re.sub( rpt_regex, rpt_repl, text )
    return text

In [3]:
# Cleaning the texts
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import LancasterStemmer


corpus = []
for i in range(0, len(df)):
#     tokens = re.sub('[^a-zA-Z]', ' ', df['text'][i])
    tokens = df['text'][i]
    tokens = tokens.lower()
    tokens = processAll(tokens)
    
    tokenizer = RegexpTokenizer("[\w']+")
    tokens = tokenizer.tokenize(tokens)
    
    tokens = [word for word in tokens if len(word) >= 3]    
    tokens = [word for word in tokens if not word in set(stopwords.words('english'))]
    
#   stemmer = PorterStemmer()
#   stemmer = LancasterStemmer()
#   tokens = [stemmer.stem(word) for word in tokens]    
    lmtzr = WordNetLemmatizer()
    tokens = [lmtzr.lemmatize(word) for word in tokens]
    
#     replacer = SpellingReplacer()
#     tokens = [replacer.replace(word) for word in tokens]
        
    tokens = ' '.join(tokens)
    corpus.append(tokens)
    

[nltk_data] Downloading package stopwords to /opt/pynb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /opt/pynb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3),token_pattern=r'\b\w+\b', min_df=5,max_features = 1500)
X = vectorizer.fit_transform(corpus).toarray()

In [5]:
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer(max_features = 1500)
# X = cv.fit_transform(corpus).toarray()

In [6]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()
# features = vectorizer.fit_transform(corpus)
# X = features.toarray()

In [7]:
i = 1000
j = 10
vectorizer.get_feature_names( )[i:i+10]

[]

In [8]:
feature_names = vectorizer.get_feature_names( )
print(feature_names[::10])

['1neg', '__hndl aww', '__hndl hey', '__hndl sorry', '__hndl wow', 'age', 'always', 'asleep', 'back work', 'best', 'bored', 'btw', 'cant wait', 'chill', 'coming', 'crazy', 'day today', 'dont', 'early', 'ever', 'fan', 'final', 'follow', 'free', 'get back', 'going bed', 'graduation', 'hanging', 'hear', 'homework', 'husband', 'ive', 'last', 'let', 'local', 'love', 'man', 'minute', 'mother', 'name', 'okay', 'party', 'play', 'project', 'ready', 'room', 'second', 'shopping', 'sister', 'soon', 'starting', 'summer', 'talking', 'theyre', 'time', 'trip', 'ugh', 'wake', 'way', 'win', 'world', 'yesterday']


In [9]:
# prop.table(table(df$sentiment))
yes_no_cols = ["sentiment"]
df[yes_no_cols] = df[yes_no_cols] == 'pos'
y = df.iloc[:, 0].values

In [10]:
from sklearn.metrics import accuracy_score as accuracy
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

# from sklearn.naive_bayes import GaussianNB
# from sklearn import linear_model
# from sklearn import tree
# from sklearn import svm
# from sklearn import ensemble
# from sklearn import neighbors

kf = KFold(len(y),n_folds=10,shuffle=True)
kf2 = StratifiedKFold(y,n_folds=10,shuffle=True)


results = cross_val_score(LogisticRegression(), X = X, y = y, scoring = "roc_auc", cv = kf)
print("LogisticRegression Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))

results = cross_val_score(SGDClassifier(loss='log', penalty='l1'), X = X, y = y, scoring = "roc_auc", cv = kf)
print("SGDClassifier Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))

# results = cross_val_score(GaussianNB(), X = X, y = y, scoring = "roc_auc", cv = kf)
# print("GaussianNB Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))
# results = cross_val_score(ensemble.RandomForestClassifier(), X = X, y = y, scoring = "roc_auc", cv = kf)
# print("RandomForest Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))
# results = cross_val_score( ensemble.GradientBoostingClassifier(), X = X, y = y,scoring = "roc_auc", cv = kf)
# print("GradientBoosting Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))


/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


LogisticRegression Accuracy: 0.751 (0.017)
SGDClassifier Accuracy: 0.723 (0.018)


In [11]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, scoring = "roc_auc", cv=10)
grid.fit(X, y)
print("Best cross-validation score: ", grid.best_score_)
print("Best parameters: ", grid.best_params_)

Best cross-validation score:  0.751816642371
Best parameters:  {'C': 1}


In [12]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
"tfidfvectorizer__ngram_range": [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]}
grid = GridSearchCV(pipe, param_grid, scoring = "roc_auc", cv=10)
grid.fit(corpus, y)
print("Best cross-validation score: ", grid.best_score_)
grid.best_params_

Best cross-validation score:  0.748532102467


{'logisticregression__C': 1, 'tfidfvectorizer__ngram_range': (1, 3)}

In [13]:
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.layers.core import Activation, Dense, Dropout
from keras.models import Sequential

def DefineModel1():    
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=615))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [14]:
model = DefineModel1()     
model.fit(X, y, epochs=50, batch_size=32, verbose=2)

Epoch 1/50
1s - loss: 0.6883 - acc: 0.5227
Epoch 2/50
0s - loss: 0.6600 - acc: 0.6357
Epoch 3/50
0s - loss: 0.6209 - acc: 0.7131
Epoch 4/50
0s - loss: 0.5721 - acc: 0.7616
Epoch 5/50
0s - loss: 0.5217 - acc: 0.7891
Epoch 6/50
0s - loss: 0.4764 - acc: 0.8026
Epoch 7/50
0s - loss: 0.4402 - acc: 0.8106
Epoch 8/50
0s - loss: 0.4089 - acc: 0.8291
Epoch 9/50
0s - loss: 0.3860 - acc: 0.8376
Epoch 10/50
0s - loss: 0.3626 - acc: 0.8511
Epoch 11/50
0s - loss: 0.3451 - acc: 0.8576
Epoch 12/50
0s - loss: 0.3277 - acc: 0.8616
Epoch 13/50
0s - loss: 0.3126 - acc: 0.8696
Epoch 14/50
0s - loss: 0.2974 - acc: 0.8731
Epoch 15/50
0s - loss: 0.2852 - acc: 0.8831
Epoch 16/50
0s - loss: 0.2719 - acc: 0.8901
Epoch 17/50
0s - loss: 0.2608 - acc: 0.8896
Epoch 18/50
0s - loss: 0.2483 - acc: 0.8951
Epoch 19/50
0s - loss: 0.2372 - acc: 0.9015
Epoch 20/50
0s - loss: 0.2280 - acc: 0.9075
Epoch 21/50
0s - loss: 0.2153 - acc: 0.9120
Epoch 22/50
0s - loss: 0.2064 - acc: 0.9150
Epoch 23/50
0s - loss: 0.1958 - acc: 0.91

In [15]:
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score as accuracy
from keras.wrappers.scikit_learn import KerasClassifier
import numpy

# create model
model = KerasClassifier(build_fn=DefineModel1, epochs=50, batch_size=32, verbose=0)
# model = KerasClassifier(build_fn=DefineModel1, epochs=50, batch_size=32)
# evaluate using 10-fold cross validation
# kf = KFold(len(y),n_folds=10,shuffle=True)
kf2 = StratifiedKFold(y,n_folds=10,shuffle=True, random_state=7)
results = cross_val_score(model, X, y,scoring = "roc_auc", cv=kf2)
print("Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))

Accuracy: 0.713 (0.038)


# Summary of roc_auc:
KerasClassifier: Accuracy: 0.713 (0.038)
LogisticRegression Accuracy: 0.751 (0.017)
SGDClassifier Accuracy: 0.723 (0.018)